In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


Please place tweets_corpus.txt in the same location as the notebook

In [156]:
import re
import pandas as pd
import numpy as np
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
import string
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
stop_words = set(stopwords.words('english')) 

In [0]:
f = open("tweets_corpus.txt")
corpus = f.read()
f.close()

In [0]:
corpus = corpus.split("\n")
corpus = corpus[:-1]

In [6]:
corpus

["81499877556760576\tBandaging up my paper-cuts , having cheesecake for dinner , and calling it a night . We're doin it big here in NYC .",
 "81500716438523904\tI haven't had any krispy kremes or strawberry trifles since I started gym * cries *",
 '81503002321616896\tBacon/cheddar slider topped w/fried egg & Blue cheese slider topped w/avocado & purple cherokee tomato',
 '81507775422791680\tNacho w/ cheese on my shirt ! Uggghhh',
 '81534165975171072\tyou aint nuffin but a piece of cheese without the corners .. in other words , you will never be a slice . BITCH .',
 '81535634019323904\tTAG_USERNAME TAG_USERNAME Mmmm ... cheese ... dreaming of a squirrel burger with cheese !',
 '81577509950459904\tMmmm cheese',
 "81582996083326976\tTAG_USERNAME 1st off I'm like 1 year younger than u , 2nd age is just a number , 3rd ima cater ur wedding wit patty n cheese",
 '81587643376336896\tRT TAG_USERNAME : I want a steak and cheese egg roll right now .',
 '81600113016971264\tthink imma eat some chee

## Problem 1
We  have  given  you  a  small  set  of  data  in  the  form  of tweets. Each line in the file begins with a document ID, followed by the text of the tweet. Implement a function to create aninverted index of these documents. You  may  use  the  approach  described  in  the  Manning  book(link  below),  or  you may come up with your own algorithm.<br>
https://nlp.stanford.edu/IR-book/html/htmledition/a-first-take-at-building-an-inverted-index-1.html

In [0]:
#Splitting the data into two lists containing document_IDs and tweets
doc_ids = []
tweets = []
documents = defaultdict(list)
for text in corpus:
    id = re.findall("^(\d{17})\t", text)[0]
    tweet = re.findall("\d{17}\t(.*)$", text)[0]
    doc_ids.append(id)
    tweets.append(tweet)
    documents[id] = tweet

In [55]:
documents

defaultdict(list,
            {'81499877556760576': "Bandaging up my paper-cuts , having cheesecake for dinner , and calling it a night . We're doin it big here in NYC .",
             '81500716438523904': "I haven't had any krispy kremes or strawberry trifles since I started gym * cries *",
             '81503002321616896': 'Bacon/cheddar slider topped w/fried egg & Blue cheese slider topped w/avocado & purple cherokee tomato',
             '81507775422791680': 'Nacho w/ cheese on my shirt ! Uggghhh',
             '81534165975171072': 'you aint nuffin but a piece of cheese without the corners .. in other words , you will never be a slice . BITCH .',
             '81535634019323904': 'TAG_USERNAME TAG_USERNAME Mmmm ... cheese ... dreaming of a squirrel burger with cheese !',
             '81577509950459904': 'Mmmm cheese',
             '81582996083326976': "TAG_USERNAME 1st off I'm like 1 year younger than u , 2nd age is just a number , 3rd ima cater ur wedding wit patty n cheese",
   

### Preprocess Text

In [0]:
#Function to preprocess text
def preprocess(text):
  #Lowercasing all the text
  text = text.lower()

  #Removing punctuation
  text = " ".join(re.sub("[!#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]", " ", text).split())

  #Tokenizing the string
  text = word_tokenize(text)

  #Removing stop words and words shorter than two characters
  text = [lemmatizer.lemmatize(word) for word in text if word not in stop_words and len(word)>2]

  #Returning document ID and prcocessed tweets
  return text

### Creating Inverted Indexes

In [0]:
#Creating an inverted index as a defaultdict
from collections import defaultdict

#Initializing the default dict
inv_indx = defaultdict(list)

#Iterating over the tweets in the corpus
for idx, text in enumerate(tweets):
    tweet = preprocess(text)

    #Iterating over each word in the tweet
    for word in tweet:
        inv_indx[word].append(doc_ids[idx])

In [0]:
inv_indx

## Problem 2
Write  a  function  to  implement  the  merge  algorithm  as discussed  in  class. Your  codeshould  allowintersecting  the  postings  of  two terms, as well asprocess simple Boolean queries. When there are multiple query terms,  make  sure  that  your  algorithm  uses  the  optimization  described  in Manningbook of performing the most restrictive intersection first.<br>
https://nlp.stanford.edu/IR-book/html/htmledition/processing-boolean-queries-1.html

### Boolean Functions

In [0]:
#Function to AND to terms
def AND(p1, p2):
    i=j=0
    ans = []
    while i<len(p1) and j<len(p2):
        if p1[i]==p2[j]:
            ans.append(p1[i])
            i+=1
            j+=1
        else:
            if p1[i]<p2[j]:
                i+=1
            else:
                j+=1
    return ans

In [0]:
#Function to OR two terms
def OR(p1, p2):
    ans = []
    i=j=0
    while i < len(p1) and j < len(p2):
        if p1[i] == p2[j]:
            ans.append(p1[i])
            i += 1
            j += 1
        else:
            if p1[i] > p2[j]:
                ans.append(p2[j])
                j += 1
            else:
                ans.append(p1[i])
                i += 1
    while i < len(p1):
        ans.append(p1[i])
        i += 1
    while j < len(p2):
        ans.append(p2[j])
        j += 1
    return ans

In [0]:
#Function to preprocess queries
def preprocess_query(text):
  #Lowercasing all the text
  text = text.lower()

  #Removing punctuation
  text = " ".join(re.sub("[!#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]", " ", text).split())

  #Tokenizing the string
  text = word_tokenize(text)

  #Removing stop words and words shorter than two characters
  text = [lemmatizer.lemmatize(word) for word in text]

  #Returning document ID and prcocessed tweets
  return text

### Query Processing

In [0]:
#Function to perform Boolean Search
def boolean_search(query):
    query = preprocess_query(query)
    terms = []
    operations = []
    for w in query:
        if w == "and" or w=="or":
            operations.append(w)
        else:
            terms.append(w)

    # for two query terms
    ans = []
    if len(query)<=3:
    p1 = query[0]
    p2 = query[2]
    if query[1]=="and":
        ans = AND(inv_indx[lemmatizer.lemmatize(p1)], inv_indx[lemmatizer.lemmatize(p2)])
    elif query[1]=="or":
        ans = OR(inv_indx[lemmatizer.lemmatize(p1)], inv_indx[lemmatizer.lemmatize(p2)])
    else:
        print("Invalid Query")
        print("Please use only AND and OR queries")

  # for more than two queries
  else:
    i = 1
    j = 0
    terms = [lemmatizer.lemmatize(x) for x in terms]
    posting_len = [len(x) for x in terms]
    p1 = inv_indx[lemmatizer.lemmatize(terms[0])]
    while i<len(terms):
        p2 = inv_indx[lemmatizer.lemmatize(terms[i])]
        if operations[j]=="and":
            p1 = AND(p1, p2)
        elif operations[j]=="or":
            p1 = OR(p1, p2)
            i+=1
            j+=1
        ans = p1
    return terms, ans

### Testing
Please enter your query after running the cell below.

In [201]:
query = input("Please Enter your query:")
terms, ans = boolean_search(query)
print("")
print("")
print("Results:")
if len(ans)>0:
    for a in ans:
        print(documents[a])

Please Enter your query:bacon and egg and cheese


Results:
Bacon/cheddar slider topped w/fried egg & Blue cheese slider topped w/avocado & purple cherokee tomato
Made myself some scrambled eggs with cheese and bacon bits
I went swimming , then ate asparagus bacon egg cheese biscuit goodness , then watched Date Night . It was ... it was good . TAG_FINAL_HASHTAGS
Cheese hashbrowns , turkey bacon , veggie tofu scramble , rice , french toast , scrambled eggs , strawberries & cantaloupe . TAG_FINAL_HASHTAGS
RT TAG_USERNAME : RT TAG_USERNAME : Pancakes , bacon , eggs w/ cheese , & hashbrown casserole on deck TAG_HASHTAGS < ~i want sum !!!!! Ok it's good too


## Problem 3
Extend the system from Problem 2 to perform simple TF-IDF  scoring  of  the  retrieved  results. There  is no  need to  worry  about  any weight normalizations.

### Calculating Term Frequencies

Reference: https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089

In [0]:
#Calculating Document Frequencies for each word
term_frequencies = {}
for i in range(len(tweets)):
    tokens = preprocess(tweets[i])
    for w in tokens:
        try:
            term_frequencies[w].add(i)
        except:
            term_frequencies[w] = {i}
for tf in term_frequencies:
    term_frequencies[tf]=len(term_frequencies[tf])

Calculating TF_IDF

In [0]:
#Calculating TF_IDF
tf_idf = {}

#Iterating over
for i in range(len(tweets)):
    tokens = preprocess(tweets[i])
    counter = Counter(tokens)
    words_count = len(tokens)
    for token in np.unique(tokens):

        #Calculating Term Frequencies
        tf = counter[token]/words_count

        #Obtaining Document Frequencies
        df = term_frequencies[token]

        #Calculating Inverse Document Frequencies
        idf = np.log(len(tweets)/(df+1))

        #Calculating TF-IDF
        tf_idf[doc_ids[i], token] = tf*idf

In [0]:
#Scoring each tweet using the sum of TF-IDF vectors
score = defaultdict(list)
for a in ans:
    score[a] = 0
    for word in terms:
        score[a]+=tf_idf[a, word]

In [0]:
import operator

#Sorting in descending order
sorted_score = sorted(score.items(), key=operator.itemgetter(1), reverse=True)

### Results

In [221]:
#Results
for i in sorted_score:
    print(documents[i[0]], i[1])

Made myself some scrambled eggs with cheese and bacon bits 0.5039838584201145
Bacon/cheddar slider topped w/fried egg & Blue cheese slider topped w/avocado & purple cherokee tomato 0.21599308218004906
I went swimming , then ate asparagus bacon egg cheese biscuit goodness , then watched Date Night . It was ... it was good . TAG_FINAL_HASHTAGS 0.18899394690754295
RT TAG_USERNAME : RT TAG_USERNAME : Pancakes , bacon , eggs w/ cheese , & hashbrown casserole on deck TAG_HASHTAGS < ~i want sum !!!!! Ok it's good too 0.18899394690754295
Cheese hashbrowns , turkey bacon , veggie tofu scramble , rice , french toast , scrambled eggs , strawberries & cantaloupe . TAG_FINAL_HASHTAGS 0.17787665591298157
